<a href="https://colab.research.google.com/github/ander-yamamoto/dio-bairesdev/blob/main/STRIDE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import json
from google.colab import userdata
from openai import OpenAI


api_key = userdata.get('key')
client = OpenAI(api_key=api_key)

In [25]:
def criar_prompt_modelo_ameacas(config):

    tipo_aplicacao = config['tipo_aplicacao']
    autenticacao = config['autenticacao']
    acesso_internet = config['acesso_internet']
    dados_sensiveis = config['dados_sensiveis']
    descricao_aplicacao = config['descricao_aplicacao']

    prompt = f"""Aja como um especialista em cibersegurança com mais de 20 anos de experiência
    utilizando a metodologia de modelagem de ameaças STRIDE para produzir modelos de ameaças
    abrangentes para uma ampla gama de aplicações. Sua tarefa é analisar o resumo do código,
    o conteúdo do README e a descrição da aplicação fornecidos para produzir uma lista de
    ameaças específicas para essa aplicação.

    Presta atenção na descrição da aplicação e nos detalhes técnicos fornecidos.

    Para cada uma das categorias do STRIDE (Falsificação de Identidade - Spoofing,
    Violação de Integridade - Tampering,
    Repúdio - Repudiation,
    Divulgação de Informações - Information Disclosure,
    Negação de Serviço - Denial of Service, e
    Elevação de Privilégio - Elevation of Privilege), liste múltiplas (3 ou 4) ameaças reais,
    se aplicável. Cada cenário de ameaça deve apresentar uma situação plausível em que a ameaça
    poderia ocorrer no contexto da aplicação.

    A lista de ameaças deve ser apresentada em formato de tabela,
    com as seguintes colunas:Ao fornecer o modelo de ameaças, utilize uma resposta formatada em JSON
    com as chaves "threat_model" e "improvement_suggestions". Em "threat_model", inclua um array de
    objetos com as chaves "Threat Type" (Tipo de Ameaça), "Scenario" (Cenário), e
    "Potential Impact" (Impacto Potencial).

    Ao fornecer o modelo de ameaças, utilize uma resposta formatada em JSON com as chaves
    "threat_model" e "improvement_suggestions".
    Em "threat_model", inclua um array de objetos com as chaves "Threat Type" (Tipo de Ameaça),
    "Scenario" (Cenário), e "Potential Impact" (Impacto Potencial).

    Em "improvement_suggestions", inclua um array de strings que sugerem quais informações adicionais
    poderiam ser fornecidas para tornar o modelo de ameaças mais completo e preciso na próxima iteração.
    Foque em identificar lacunas na descrição da aplicação que, se preenchidas, permitiriam uma
    análise mais detalhada e precisa, como por exemplo:
    - Detalhes arquiteturais ausentes que ajudariam a identificar ameaças mais específicas
    - Fluxos de autenticação pouco claros que precisam de mais detalhes
    - Descrição incompleta dos fluxos de dados
    - Informações técnicas da stack não informadas
    - Fronteiras ou zonas de confiança do sistema não especificadas
    - Descrição incompleta do tratamento de dados sensíveis
    - Detalhes sobre
    Não forneça recomendações de segurança genéricas — foque apenas no que ajudaria a criar um
    modelo de ameaças mais eficiente.

    TIPO DE APLICAÇÃO: {tipo_aplicacao}
    MÉTODOS DE AUTENTICAÇÃO: {autenticacao}
    EXPOSTA NA INTERNET: {acesso_internet}
    DADOS SENSÍVEIS: {dados_sensiveis}
    RESUMO DE CÓDIGO, CONTEÚDO DO README E DESCRIÇÃO DA APLICAÇÃO: {descricao_aplicacao}

    Exemplo de formato esperado em JSON:

    {{
      "threat_model": [
        {{
          "Threat Type": "Spoofing",
          "Scenario": "Cenário de exemplo 1",
          "Potential Impact": "Impacto potencial de exemplo 1"
        }},
        {{
          "Threat Type": "Spoofing",
          "Scenario": "Cenário de exemplo 2",
          "Potential Impact": "Impacto potencial de exemplo 2"
        }}
        // ... mais ameaças
      ],
      "improvement_suggestions": [
        "Por favor, forneça mais detalhes sobre o fluxo de autenticação entre os componentes para permitir uma análise melhor de possíveis falhas de autenticação.",
        "Considere adicionar informações sobre como os dados sensíveis são armazenados e transmitidos para permitir uma análise mais precisa de exposição de dados.",
        // ... mais sugestões para melhorar o modelo de ameaças
      ]
    }}"""

    return prompt

In [26]:
def gerar_modelo_ameacas(imagem, config):

  prompt = criar_prompt_modelo_ameacas(config)

  chat_prompt = [
      {"role": "system", "content": "Você é uma IA especialista em cibersegurança, que analisa desenhos de arquitetura."},
      {"role": "user"
        , "content": [
          {"type": "text"
            , "text": prompt
            },
          {
              "type": "image_url"
            ,  "image_url": {"url": imagem}
            },
          {"type": "text",
            "text": "Por favor, analise a imagem e o texto acima e forneça um modelo de ameaças detalhado."
            }]
  }]
  response = client.chat.completions.create(
    model="gpt-4o",
    messages=chat_prompt,
    temperature=0.7,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    max_completion_tokens=1500,
  )

  return response.choices[0].message.content

In [31]:
imagem="https://www.slideteam.net/wp/wp-content/uploads/2023/08/2Arquitetura-do-Sistema-Operacional-de-Data-Warehouse.png"

config = {
    'tipo_aplicacao':"data science",
    'autenticacao':"user",
    'acesso_internet':"sim",
    'dados_sensiveis':"sim",
    'descricao_aplicacao':"Pipeline de dados desde a aquisição até o consumo",
}


resposta = gerar_modelo_ameacas(imagem, config)
print(resposta)


```json
{
  "threat_model": [
    {
      "Threat Type": "Spoofing",
      "Scenario": "Um atacante obtém credenciais de usuário válidas e se passa por um usuário legítimo para acessar dados sensíveis no data warehouse.",
      "Potential Impact": "Acesso não autorizado a dados sensíveis, resultando em violação de privacidade e possível roubo de dados."
    },
    {
      "Threat Type": "Spoofing",
      "Scenario": "Um atacante intercepta o tráfego entre o usuário e a aplicação analítica para falsificar a identidade do usuário.",
      "Potential Impact": "Execução de consultas maliciosas, modificação de dados e acesso não autorizado a informações."
    },
    {
      "Threat Type": "Tampering",
      "Scenario": "Um invasor modifica os dados durante o processo de ETL, alterando informações críticas antes de serem armazenadas no data warehouse.",
      "Potential Impact": "Dados corrompidos ou manipulados que podem levar a decisões de negócio erradas."
    },
    {
      "Threat Type"

In [35]:
imagem = "https://cdn-wcsm.alura.com.br/2025/04/imagem12-16.jpg"
config = {
    'tipo_aplicacao':"aplicativo",
    'autenticacao':"user",
    'acesso_internet':"sim",
    'dados_sensiveis':"sim",
    'descricao_aplicacao':"aplicativo",
}


resposta = gerar_modelo_ameacas(imagem, config)
print(resposta)

```json
{
  "threat_model": [
    {
      "Threat Type": "Spoofing",
      "Scenario": "Um atacante falsifica credenciais de usuário para acessar o sistema de pedidos.",
      "Potential Impact": "Acesso não autorizado a dados de pedidos e informações pessoais dos clientes."
    },
    {
      "Threat Type": "Spoofing",
      "Scenario": "Um invasor intercepta a comunicação entre a interface do usuário e o sistema de pagamentos.",
      "Potential Impact": "Transações fraudulentas e comprometimento de dados financeiros."
    },
    {
      "Threat Type": "Tampering",
      "Scenario": "Um atacante altera dados de pedidos no banco de dados.",
      "Potential Impact": "Pedidos incorretos e possível perda financeira."
    },
    {
      "Threat Type": "Tampering",
      "Scenario": "Manipulação do fluxo de dados entre o sistema de análise de dados e o banco de dados.",
      "Potential Impact": "Relatórios imprecisos e decisões baseadas em dados errados."
    },
    {
      "Threat Type"